### Imports and setups 
Below are the imports and setups needed for the lab. 

In [2]:
# All imports
from tkinter import Image
import numpy as np
from scipy.signal import convolve2d, correlate2d
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt


def kmeans_segm(image, K, L, seed):
    """
    Implement k-means clustering:
    - Input: image : image input
             K : number of cluster centres
             L : number of iterations
             seed : to initialize random generator
    - Output: segmentation : segmented image with a colour index per pixel
              centers : final cluster centres in 3D colour space
    """
    # This is used to flatten the image
    Ivec = np.reshape(image, (-1, 3))

    # Apply k-means clustering algorithm
    # Initialize cluster centers randomly
    centers = Ivec[np.random.seed(seed), :]

    # Initialize nearest_clustery
    nearest_cluster = np.zeros_like(Ivec[:, 0])

    for _ in range(L):
        # Assign pixels to the nearest cluster center by using pyhtagoras equation
        distances = np.sqrt(
            ((Ivec[:, np.newaxis, :] - centers[np.newaxis, :, :]) ** 2).sum(axis=2)
        )
        nearest_cluster = np.argmin(distances, axis=1)

        # Update cluster centers
        for k in range(K):
            centers[k] = Ivec[nearest_cluster == k].mean(axis=0)

    segmentation = np.reshape(nearest_cluster, (-1, 3))
    return segmentation, centers

def open_image(name):
    """
    Implement a function that opens an image using Pillow. It should return a numpy array with the image data,  should be a floating point
    """
    image = Image.open(name)
    image = np.array(image).astype(np.float32) 
    return image



### Question 1:
Create a function in python that implements the K-means clustering algorithm. 

**Question 1: How did you initialize the clustering process and why do you believe this was a good method of doing it?**
